1. Install and import required python packages

In [1]:
!pip install --upgrade pip
!pip install -r requirements.txt
import os
import requests
import httpx
import json
import google.auth.transport.requests
from google.oauth2 import gdch_credentials
from openai import OpenAI

# Disable self-signed cert warning for test environment. Remove this for production environment
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

2. Setup environments  

In [2]:
# Use the project folder as working directory
from pathlib import Path
WORK_DIR = Path().cwd()
# If running from python script rather than notebook, the following code is more reliable
# WORK_DIR = os.path.dirname(os.path.abspath(__file__))

# Service account key file
SA_KEY_FILE = "sa-key.json"
SA_KEY_FILE = os.path.join(WORK_DIR, SA_KEY_FILE)

# ORG_NAME = "lancer-org1"
ZONE_NAME = "us-east38-a"
# DOMAIN = "google.gdch.test"

# for Gemini on GDCag, the inference api endpoint is the FQDN of inference gateway
ENDPOINT = "inference-gateway-aics-system.aics.lancer-org1.us-east38-a.google.gdch.test:443"
# User project is the GDC-ag project where your client code is deployed
USER_PROJECT="gemini-test-38"

MODEL_NAME = "gemini-1.5-flash-002"

print(f"WORK_DIR = {WORK_DIR}")
print(f"SA_KEY_FILE = {SA_KEY_FILE}")
print(f"ZONE_NAME = {ZONE_NAME}")
print(f"ENDPOINT = {ENDPOINT}")
print(f"USER_PROJECT = {USER_PROJECT}")
print(f"MODEL_NAME = {MODEL_NAME}")

WORK_DIR = /usr/local/google/home/danielxia/PycharmProjects/ge2e_sample
SA_KEY_FILE = /usr/local/google/home/danielxia/PycharmProjects/ge2e_sample/sa-key.json
ZONE_NAME = us-east38-a
ENDPOINT = inference-gateway-aics-system.aics.lancer-org1.us-east38-a.google.gdch.test:443
USER_PROJECT = gemini-test-38
MODEL_NAME = gemini-1.5-flash-002


3. Define a class to wrap up openai api client  
A service account key with sufficient permission is required to access gemini API on GDC  
*Pay attention to the audience when retrieve STS token.*  
    *For ***Openai(HTTP)*** request, the audience should ***EXCLUDE*** the port*  
    *For ***GRPC*** request, the audience should ***INCLUDE*** the port*  
    e.g. if the endpoint is `inference-gateway-aics-system.aics.lancer-org1.us-east38-a.google.gdch.test:443`  
   The HTTP audience is `https://inference-gateway-aics-system.aics.lancer-org1.us-east38-a.google.gdch.test`,  
   while the GRPC audience is `https://inference-gateway-aics-system.aics.lancer-org1.us-east38-a.google.gdch.test:443`

In [3]:
class GdcOpenaiClient:

    def __init__(self, endpoint, zone_name, userproject, sa_key_file,
                 verify_cert: bool = True, timeout: int = 300):
        self.sa_key_file = sa_key_file
        self.zone_name = zone_name
        self.userproject = userproject
        self.endpoint = f"https://{endpoint}"
        # Openai API is based on HTTP, so the audience does not include port
        self.audience = f"https://{endpoint.split(":")[0]}"
        # Use the service account key file and audience to build credentials
        self.credentials = gdch_credentials.ServiceAccountCredentials.from_service_account_file(
            self.sa_key_file).with_gdch_audience(self.audience)
        # If using self-signed certs, set verify_cert to False
        # For production environment, set verify_cert to True
        self.verify_cert = verify_cert
        self.token = None
        self.timeout = timeout

    def get_token(self):
        # The generated STS token will be used in HTTP request header
        session = requests.Session()
        session.verify = self.verify_cert
        request = google.auth.transport.requests.Request(session=session)
        self.credentials.refresh(request)
        self.token = self.credentials.token

    def send_request(self, req_content, model: str = "gemini-1.5-flash-002", stream: bool = True):
        
        # update model and stream setting in the request content
        req_content["model"] = model
        req_content["stream"] = stream
        
        # construct the base url of request path
        base_url = f"{self.endpoint}/v1/projects/{self.userproject}/locations/{self.zone_name}"
        
        # use a http 2.0 client.
        httpx_client = httpx.Client(
            http2=True, follow_redirects=True, verify=self.verify_cert,
            timeout=self.timeout if self.timeout > 0 else None
        )
        
        self.get_token()
        openai_client = OpenAI(base_url=base_url, http_client=httpx_client, api_key=self.token)
        response = openai_client.chat.completions.create(
            **req_content,
            extra_headers={"x-goog-user-project": f"projects/{self.userproject}"}
        )
        usage = {}
        if stream:
            for chunk in response:
                if not chunk.choices:
                    continue
                rcv_content = chunk.choices[0].delta.content
                if chunk.usage:
                    usage = chunk.usage.to_dict()
                yield rcv_content, usage
        else:
            rcv_content = response.choices[0].message.content
            if response.usage:
                usage = response.usage
            yield rcv_content, usage

4. Inferencing with prepared json request files  
Prepared json request file is saved in [contents](contents) folder. The request files may contain the combination of text, image and audio. The media files, e.g. image and audio, are base64 encoded and embedded in the json request file.  
Please refer to the folder [media_files](media_files) for the original files  


* Create a client.  
  Repalce `SA_KEY_FILE` with your own service account key

In [4]:
ai_client = GdcOpenaiClient(
    endpoint = ENDPOINT,
    zone_name = ZONE_NAME,
    userproject = USER_PROJECT,
    sa_key_file = SA_KEY_FILE,
    verify_cert = False
)

* Text input with streaming output

In [5]:
TEXT_INPUT = json.load(open(os.path.join(WORK_DIR, "contents", "text.json"), "r"))
response = ai_client.send_request(TEXT_INPUT, model=MODEL_NAME, stream=True)
for chunk in response:
    print(chunk[0], end="")
print("")

I am doing well, thank you for asking! How are you today?



* Text input with unary output

In [6]:
response = ai_client.send_request(TEXT_INPUT, model=MODEL_NAME, stream=False)
for chunk in response:
    print(chunk[0], end="")
print("")

I am doing well, thank you for asking!  How are you today?



* Image input with unary output  

![image](media_files/image.jpg)

In [7]:
IMAGE_INPUT = json.load(open(os.path.join(WORK_DIR, "contents", "image.json"), "r"))
response = ai_client.send_request(IMAGE_INPUT, model=MODEL_NAME, stream=False)
for chunk in response:
    print(chunk[0], end="")
print("")

That's a **Lisbon tram** (eléctrico) in Lisbon, Portugal.  Specifically, it looks like one of the older, iconic models that run on the city's historic tram lines.  The yellow paint and overall style are characteristic of these trams.



* Audio input with streaming output  

[audio.mp3](media_files/audio.mp3)

In [8]:
AUDIO_INPUT = json.load(open(os.path.join(WORK_DIR, "contents", "audio.json"), "r"))
response = ai_client.send_request(AUDIO_INPUT, model=MODEL_NAME, stream=True)
for chunk in response:
    print(chunk[0], end="")
print("")

This is a speech given by Martin Luther King Jr.  The speaker begins by stating that the current event will go down in history as the greatest demonstration for freedom in the nation's history. He then references the Emancipation Proclamation, signed 100 years prior, which brought hope to millions of slaves, but notes that 100 years later, the Negro is still not free, and remains burdened by segregation and discrimination.  He speaks of a dream where one day the sons of former slaves and slave owners will sit together, and where the state of Mississippi will be transformed into an oasis of freedom and justice. He dreams of a day when his children will be judged by the content of their character, not the color of their skin. The speech then continues with a powerful call for freedom and equality across the United States, invoking imagery of freedom ringing from mountains and hills across different states. The speech culminates in a vision of a day when black and white men, Jews and Gent